In [ ]:
import os
import numpy as np
import pandas as pd
from flask import Flask, request
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
from collections.abc import Iterable
import pickle
from flask_cors import CORS
import math
print('The scikit-learn version is {}.'.format(sklearn.__version__))
app = Flask(__name__)
CORS(app)

@app.route('/')
def hello():
    return "Hello An!"


def weightedRating(x):
    v = x['Rating']
    R = x['AggregatedRating']
    return (v / (v + 51.0) * R) + (51.0 / (51.0 + v) * 4.56)

categoryBasedRecommender = pickle.load(open('categoryBasedRecommender.pkl', 'rb'))
metadataBasedRecommender = pickle.load(open('metadataBasedRecommender.pkl', 'rb'))

datasetDumpAlgolia = pickle.load(open('datasetDump.pkl', 'rb'))
print(datasetDumpAlgolia.shape)

# weighted popularity model
#qualifiedRecipes = pickle.load(open('qualifiedRecipes.pkl', 'rb'))


qualifiedRecipes = pickle.load(open('qualifiedRecipesCollaborative.pkl', 'rb'))
qualifiedRecipes['score'] = qualifiedRecipes.apply(weightedRating, axis=1)
df_rating = pd.DataFrame(qualifiedRecipes.groupby(by = ['RecipeId'])['score'].agg('mean')) 

filtered_ratings = pickle.load(open('filtered_ratings.pkl', 'rb'))
collaborativeMatrix = filtered_ratings.pivot(index = 'AuthorId', columns ='RecipeId', values = 'Rating').fillna(0)

recipe_metadata = qualifiedRecipes[['RecipeId','Name','RecipeCategory']]
recipe_data = filtered_ratings.merge(recipe_metadata,on='RecipeId')
knnMatrix = recipe_data.pivot(index = 'RecipeId', columns ='AuthorId', values = 'Rating').fillna(0)
    

@app.route('/popularity')
def weightedRatingPopularity():
    return qualifiedRecipes.to_json()


# category - got the data in weightedPopularRecipes and now creating the route function
@app.route('/category', methods=['POST'])
def getCategoryWiseRecommendations():
    data = request.get_json()
    RecipeCategory = data['keyword']
    percentile = 0.85
    categorySpecificRecipes = categoryBasedRecommender[categoryBasedRecommender['Category'] == RecipeCategory]
    RatingCounts = categorySpecificRecipes[categorySpecificRecipes['Rating'].notnull()]['Rating'].astype('int')
    RatingAverages = categorySpecificRecipes[categorySpecificRecipes['AggregatedRating'].notnull()][
        'AggregatedRating'].astype('int')
    C = RatingAverages.mean()
    m = RatingCounts.quantile(percentile)

    qualified = categorySpecificRecipes[
        (categorySpecificRecipes['Rating'] >= m) & (categorySpecificRecipes['Rating'].notnull()) & (
            categorySpecificRecipes['AggregatedRating'].notnull())][
        ['RecipeId', 'Name', 'Rating', 'AggregatedRating', 'ReviewCount']]
    qualified['Rating'] = qualified['Rating'].astype('int')
    qualified['AggregatedRating'] = qualified['AggregatedRating'].astype('int')

    qualified['wr'] = qualified.apply(
        lambda x: (x['Rating'] / (x['Rating'] + m) * x['AggregatedRating']) + (m / (m + x['Rating']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(12)
    print(qualified.to_json())
    return qualified.to_json()


# metadata
termFrequency2 = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0.0, stop_words='english')
tfidMatrix2 = termFrequency2.fit_transform(metadataBasedRecommender['Metadata'])
cosineSimilarity2 = linear_kernel(tfidMatrix2, tfidMatrix2)

metadataBasedRecommender = metadataBasedRecommender.reset_index()
names2 = metadataBasedRecommender[['RecipeId', 'Name']]
indices2 = pd.Series(metadataBasedRecommender.index, index=metadataBasedRecommender['Name'])


@app.route('/metadata', methods=['POST'])
def getMetadataBasedRecommendations():
    data = request.get_json()
    title = data['keyword']
    idx = indices2[title]
    if isinstance(idx, Iterable):
        for i in idx:
            similarityScores = sorted(list(enumerate(cosineSimilarity2[i])), key=lambda x: x[1], reverse=True)[1:13]
            break
    else:
        similarityScores = sorted(list(enumerate(cosineSimilarity2[idx])), key=lambda x: x[1], reverse=True)[1:13]
    recipeIndices = [i[0] for i in similarityScores]
    return names2.iloc[recipeIndices].to_json()


# content with ratings considered
contentBasedRecommender = metadataBasedRecommender


@app.route('/content', methods=['POST'])
def getContentBasedRecommendations():
    data = request.get_json()
    title = data['keyword']
    idx = indices2[title]

    if isinstance(idx, Iterable):
        recipeDump = pd.DataFrame()
        for i in idx:
            cnt = 0
            similarityScores = sorted(list(enumerate(cosineSimilarity2[i])), key=lambda x: x[1], reverse=True)[1:26]
            recipeIndices = [i[0] for i in similarityScores]
            recipeSimilarity = [i[1] for i in similarityScores]
            
            contentBasedRecommendedRecipes = contentBasedRecommender.iloc[recipeIndices][
                ['RecipeId', 'Name', 'Rating', 'AggregatedRating']]
            ratingCounts = contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['Rating'].notnull()][
                'Rating'].astype('int')
            ratingAverages = \
            contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['AggregatedRating'].notnull()][
                'AggregatedRating'].astype('int')
            C = ratingAverages.mean()
            m = ratingCounts.quantile(0.60)
            contentBasedRecommendedRecipes['Similarity'] = recipeSimilarity
            qualifiedRecipes2 = contentBasedRecommendedRecipes[(contentBasedRecommendedRecipes['Rating'] >= m) & (
                contentBasedRecommendedRecipes['Rating'].notnull()) & (contentBasedRecommendedRecipes[
                                                                           'AggregatedRating'].notnull())]
            qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
            qualifiedRecipes2['AggregatedRating'] = qualifiedRecipes2['AggregatedRating'].astype('int').head(12)
            if cnt == 0:
                recipeDump = qualifiedRecipes2
            else:
                recipeDump.append(qualifiedRecipes2, ignore_index=True)
            cnt += 1
        recipeDump = recipeDump.sort_values('Similarity', ascending=False)
        recipeDump = recipeDump.reset_index()
        return recipeDump.head(12).to_json()
 
    else:
        similarityScores = sorted(list(enumerate(cosineSimilarity2[idx])), key=lambda x: x[1], reverse=True)[1:26]
        recipeIndices = [i[0] for i in similarityScores]
        recipeSimilarity = [i[1] for i in similarityScores]
        
        contentBasedRecommendedRecipes = contentBasedRecommender.iloc[recipeIndices][
            ['RecipeId', 'Name', 'Rating', 'AggregatedRating']]
        ratingCounts = contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['Rating'].notnull()][
            'Rating'].astype('int')
        ratingAverages = contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['AggregatedRating'].notnull()][
            'AggregatedRating'].astype('int')
        C = ratingAverages.mean()
        m = ratingCounts.quantile(0.60)
        contentBasedRecommendedRecipes['Similarity'] = recipeSimilarity
        qualifiedRecipes2 = contentBasedRecommendedRecipes[
            (contentBasedRecommendedRecipes['Rating'] >= m) & (contentBasedRecommendedRecipes['Rating'].notnull()) & (
                contentBasedRecommendedRecipes['AggregatedRating'].notnull())]
        qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
        qualifiedRecipes2['AggregatedRating'] = qualifiedRecipes2['AggregatedRating'].astype('int')
        qualifiedRecipes2 = qualifiedRecipes2.sort_values(by='Similarity', ascending=False)
        qualifiedRecipes2 = qualifiedRecipes2.reset_index()
        return qualifiedRecipes2.head(12).to_json()

@app.route('/collaborative', methods=['POST'])
def getCollaborativeRecommendations():
    data = request.json
    if 'likedRecipeList' in data:
        likedRecipeList = data['likedRecipeList']
        return getCollaborativeRecommendations(likedRecipeList)  
    return pd.DataFrame().to_json()

def getCollaborativeRecommendations(recipe_ids):
    recommendItemLength = 4
    
    for recipe_id in recipe_ids:
        if recipe_id not in collaborativeMatrix.columns:
            # remove recipe that doesnt have more than {threshold} count of reviews from participating in correlation matching
            recipe_ids.remove(recipe_id)

    recommendRecipeIndices = []
    for recipe_id in recipe_ids:
        users_rating = collaborativeMatrix[recipe_id]
        similar_recipes = collaborativeMatrix.corrwith(users_rating, method='pearson')
        similar_recipes = pd.DataFrame(similar_recipes,columns=['correlation'])
        result_recipes = similar_recipes.join(df_rating['score']).sort_values(by='correlation', ascending=False)
        result_recipes = result_recipes[result_recipes['score'] > 4].sort_values(by = 'correlation', ascending = False) 
        recommendRecipeIndices.extend(result_recipes.iloc[1:recommendItemLength + 1].index.tolist())

    filtered_rows = pd.DataFrame(columns=(qualifiedRecipes.columns.tolist() + ['SourceRecipeId']))
    
    for k in range(len(recommendRecipeIndices)):
        for i in range(len(qualifiedRecipes)):
            if qualifiedRecipes.iloc[i]['RecipeId'] == recommendRecipeIndices[k]:
                filtered_rows = filtered_rows.append(qualifiedRecipes.iloc[i], ignore_index=True)
                filtered_rows.loc[filtered_rows.index[-1], 'SourceRecipeId'] = recipe_ids[math.floor(k / recommendItemLength)]
                break
    filtered_rows = filtered_rows.sort_values(by=['SourceRecipeId', 'score'], ascending=[True, False])
    print(filtered_rows)
    filtered_rows = filtered_rows.reset_index()
    returnValue = filtered_rows[['RecipeId', 'Name', 'Images', 'TotalTime', 'RecipeCategory', 'SourceRecipeId', 'score']]
    return returnValue.to_json()

#print(getCollaborativeRecommendations([56, 10744]))

@app.route('/knn', methods=['POST'])
def getKNNRecommendations():
    data = request.json
    if 'likedRecipeList' in data:
        likedRecipeList = data['likedRecipeList']
        return knn_recommender(likedRecipeList)  
    return pd.DataFrame().to_json()

def knn_recommender(recipe_ids, matrix = knnMatrix, n_recs = 10, n_result = 4):
    n_recs = n_result * len(recipe_ids)
    cf_knn_model= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
    cf_knn_model.fit(matrix)
    cf_recs = pd.DataFrame(columns=(qualifiedRecipes.columns.tolist() + ['SourceRecipeId', 'Distance']))
    
    for recipe_id in recipe_ids:
        recipe_index = process.extractOne(str(recipe_id),qualifiedRecipes['RecipeId'])[2]
        distances, indices = cf_knn_model.kneighbors(matrix.iloc[recipe_index,:].values.reshape(1,-1), n_neighbors=n_recs)
        recipe_rec_ids = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        iterNo = range(len(recipe_rec_ids))
        for i in iterNo:
            if i == n_result:
                break
            index_to_append = recipe_rec_ids[i][0]
            if not any(cf_recs['RecipeId'] == qualifiedRecipes.iloc[index_to_append]['RecipeId']):
                cf_recs = cf_recs.append(qualifiedRecipes.iloc[index_to_append], ignore_index=True)
                cf_recs.loc[cf_recs.index[-1], 'SourceRecipeId']  = recipe_id
                cf_recs.loc[cf_recs.index[-1], 'Distance']  = recipe_rec_ids[i][1]
            else:
                iterNo = iterNo + 1
    
    cf_recs = cf_recs[['RecipeId', 'Name', 'Images', 'TotalTime', 'RecipeCategory', 'SourceRecipeId', 'Distance']]
    return cf_recs.to_json()

#print(knn_recommender([56, 26499]))

@app.route('/datasetDump')
def getDatasetDump():
    return datasetDumpAlgolia.to_json()


if __name__ == "__main__":
    osPort = os.getenv("PORT")
    if osPort == None:
        port = 5000
    else:
        port = int(osPort)
    app.run(host='0.0.0.0', port=port)

# class ModelDataHolder:
#     def __init__(self, matrix)

C:\Users\CY\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


The scikit-learn version is 1.2.2.
(9000, 26)
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.94:5000
Press CTRL+C to quit
127.0.0.1 - - [05/May/2024 22:56:50] "GET /datasetDump HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 22:56:55] "GET /popularity HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 22:57:06] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 22:57:06] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 22:57:06] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 22:57:06] "POST /category HTTP/1.1" 200 -


{"RecipeId":{"5873":11149,"6392":11808,"5899":11181,"4035":8617,"3588":7949,"5974":11286,"4549":9265,"4638":9403,"5938":11237,"5815":11078,"5061":10134,"4048":8634},"Name":{"5873":"Chili Con Carne With Beans","6392":"Pasta with Mushroom Garlic Sauce","5899":"Easy French Onion Soup","4035":"Weeknight Tuna Noodle Casserole","3588":"Tortellini Ai Formaggi With Prosciutto and Peas","5974":"Cranberry Chicken","4549":"Tex-Mex Cornbread Salad","4638":"Taco Soup","5938":"Macaroni Tuna Casserole","5815":"Soupa Avgolemono (Egg-Lemon Soup)","5061":"Ground Beef & Bacon Stuffed Bell Peppers","4048":"Pasta with Olive Oil And Garlic"},"Rating":{"5873":286,"6392":101,"5899":86,"4035":74,"3588":50,"5974":48,"4549":39,"4638":35,"5938":33,"5815":32,"5061":28,"4048":26},"AggregatedRating":{"5873":5,"6392":5,"5899":5,"4035":5,"3588":5,"5974":5,"4549":5,"4638":5,"5938":5,"5815":5,"5061":5,"4048":5},"ReviewCount":{"5873":300.0,"6392":103.0,"5899":90.0,"4035":75.0,"3588":51.0,"5974":48.0,"4549":39.0,"4638":36

127.0.0.1 - - [05/May/2024 22:58:39] "GET /datasetDump HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 22:58:42] "GET /popularity HTTP/1.1" 200 -
C:\Users\CY\AppData\Local\Temp\ipykernel_17732\2023128395.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
C:\Users\CY\AppData\Local\Temp\ipykernel_17732\2023128395.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualifiedRecipes2['AggregatedRating'] = qualifiedRecipes2['AggregatedRating

...

   RecipeId                                               Name  \
1    113983                               Classic Saffron Rice   
0     50004                                        Lil Smokies   
3    167914  The Best Fluffiest Buttermilk Pancakes on the ...   
2     10958                        Cheating Scalloped Potatoes   
11    25690                                           Pancakes   
10    77397     Kittencal's 5-Minute Cinnamon Flop Brunch Cake   
9     66241                                Buttermilk Pancakes   
8     73033         Allrighty then Scrambled Eggs - Paula Deen   
7     68955                             Japanese Mum's Chicken   
6     16884                           Awesome Bacon-Tomato Dip   
4    105726  Kittencal's Parmesan Garlic Bread or Garlic Toast   
5     22542                    Curried Cranberry Chicken Salad   

         RecipeCategory AggregatedRating Rating  ReviewCount  \
1            White Rice                5     88         91.0   
0          Lu

127.0.0.1 - - [05/May/2024 23:03:14] "GET /popularity HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 23:03:15] "POST /content HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 23:03:16] "POST /content HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 23:03:17] "POST /content HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 23:03:18] "POST /content HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 23:03:21] "OPTIONS /collaborative HTTP/1.1" 200 -
C:\Users\CY\AppData\Local\Temp\ipykernel_17732\2023128395.py:202: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_rows = filtered_rows.append(qualifiedRecipes.iloc[i], ignore_index=True)
C:\Users\CY\AppData\Local\Temp\ipykernel_17732\2023128395.py:202: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_rows = filtered_rows.append(qualifiedRecipes.iloc[i], ignore_index=True)
C:\Users\CY\App

  RecipeId                                               Name  \
1   113983                               Classic Saffron Rice   
0    50004                                        Lil Smokies   
3   167914  The Best Fluffiest Buttermilk Pancakes on the ...   
2    10958                        Cheating Scalloped Potatoes   
7    68955                             Japanese Mum's Chicken   
6    16884                           Awesome Bacon-Tomato Dip   
4   105726  Kittencal's Parmesan Garlic Bread or Garlic Toast   
5    22542                    Curried Cranberry Chicken Salad   

        RecipeCategory AggregatedRating Rating  ReviewCount  \
1           White Rice                5     88         91.0   
0         Lunch/Snacks                5     66         69.0   
3            Breakfast                5     56         57.0   
2         Lunch/Snacks                5     53         54.0   
7  Chicken Thigh & Leg                5    910        973.0   
6                 Pork              

127.0.0.1 - - [05/May/2024 23:04:05] "POST /content HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 23:04:16] "POST /content HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 23:04:18] "OPTIONS /knn HTTP/1.1" 200 -
C:\Users\CY\AppData\Local\Temp\ipykernel_17732\2023128395.py:237: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cf_recs = cf_recs.append(qualifiedRecipes.iloc[index_to_append], ignore_index=True)
C:\Users\CY\AppData\Local\Temp\ipykernel_17732\2023128395.py:237: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cf_recs = cf_recs.append(qualifiedRecipes.iloc[index_to_append], ignore_index=True)
C:\Users\CY\AppData\Local\Temp\ipykernel_17732\2023128395.py:237: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cf_recs = cf_recs.append(q

  RecipeId                                               Name  \
1   113983                               Classic Saffron Rice   
0    50004                                        Lil Smokies   
3   167914  The Best Fluffiest Buttermilk Pancakes on the ...   
2    10958                        Cheating Scalloped Potatoes   
7    68955                             Japanese Mum's Chicken   
6    16884                           Awesome Bacon-Tomato Dip   
4   105726  Kittencal's Parmesan Garlic Bread or Garlic Toast   
5    22542                    Curried Cranberry Chicken Salad   

        RecipeCategory AggregatedRating Rating  ReviewCount  \
1           White Rice                5     88         91.0   
0         Lunch/Snacks                5     66         69.0   
3            Breakfast                5     56         57.0   
2         Lunch/Snacks                5     53         54.0   
7  Chicken Thigh & Leg                5    910        973.0   
6                 Pork              